In [203]:
import cv2
import numpy as np

In [246]:
def view_image(image):
    cv2.namedWindow('Display', cv2.WINDOW_NORMAL)
    cv2.imshow('Display', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#Here I will get the red lines (top of the step)
def get_red_lines(image):
    hsv_stairs = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    red_low = np.array([-10,100,100])
    red_high = np.array([10,255,255])
    mask = cv2.inRange(hsv_stairs, red_low, red_high)
    
    res = cv2.bitwise_and(hsv_stairs, hsv_stairs, mask = mask)

    res_bgr = cv2.cvtColor(res, cv2.COLOR_HSV2BGR)
    res_gray = cv2.cvtColor(res_bgr, cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(res_gray,50,150,apertureSize = 3)

    return cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=50,maxLineGap=10)

#Here I will get all the edges of the image
def get_all_lines(image):
    res_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(res_gray,50,150,apertureSize = 3)
    return cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=50,maxLineGap=10)

#Here I will get the line equation of each line that I got on the last step
def lines_equation(lines):
    return list(map(lambda line : np.polyfit([line[0][0],line[0][2]], [line[0][1],line[0][3]], 1), lines))

#This is for filtering all the horizontal lines
def horizontal_lines(lines):
    return list(filter(lambda x : abs(x[0]) < 1e-15, lines))

def get_step_top(lines_filtered):
    value = 0
    for line in lines_filtered:
        if line[1] - value>5:
            value = line[1]
    return value

def get_step_bottom(lines_filtered_all, value):
    value_2 = value
    for line in lines_filtered_all:
        if 1 < line[1] - value and line[1] - value < 40 and line[1]>value:
            value_2 = line[1]
    return value_2

def draw_lines(y, image):
    cv2.line(image, (0, y), (image.shape[1], y), (255,0,0), 2)
    return

Website with the humanoid camera especifications:

https://www.raspberrypi.org/documentation/hardware/camera/

In [250]:
stairs = cv2.imread('stairs_01.jpeg')

red_lines = get_red_lines(stairs)
all_lines = get_all_lines(stairs)

red_lines_equation = lines_equation(red_lines)
all_lines_equation = lines_equation(all_lines)

red_lines_horizontal = horizontal_lines(red_lines_equation)
all_lines_horizontal = horizontal_lines(all_lines_equation)

top_y = get_step_top(red_lines_horizontal)
bottom_y = get_step_bottom(all_lines_horizontal, top_y)

height_pixels = "{:.2f}".format(bottom_y - top_y)


draw_lines(int(top_y), stairs)
draw_lines(int(bottom_y), stairs)

view_image(stairs)
print("Altura em pixels:", height_pixels)

Altura em pixels: 14.00


In [ ]:
pinhole camera model 